In [12]:
import pandas as pd
import numpy as np
from keras import Model, Sequential
from keras.layers import Embedding, Input, Flatten, Dot, Add
from keras.regularizers import l2
from keras.optimizers import Adam
import tensorflow.keras.backend as K 

from datetime import datetime

In [13]:
ratings = pd.read_csv('../data/scores.csv')
df_users = pd.read_csv('../data/usuarios.csv')
df_movies = pd.read_csv('../data/peliculas.csv')

df_movies.loc[df_movies['IMDB URL'].isna(), 'IMDB URL'] = ''

u_unique = ratings.user_id.unique()
user2Idx = {o:i+1 for i,o in enumerate(u_unique)}

m_unique = ratings.movie_id.unique()
movie2Idx = {o:i+1 for i,o in enumerate(m_unique)}

ratings.user_id = ratings.user_id.apply(lambda x: user2Idx[x])

ratings.movie_id = ratings.movie_id.apply(lambda x: movie2Idx[x])

ratings.head()

id  user_id  movie_id  rating                 Date
0   0        1         1       3  1997-12-04 15:55:49
1   1        2         2       3  1998-04-04 19:22:22
2   2        3         3       1  1997-11-07 07:18:36
3   3        4         4       2  1997-11-27 05:02:03
4   4        5         5       1  1998-02-02 05:33:16

In [14]:
from sklearn.model_selection import train_test_split
ratings_train, ratings_val = train_test_split(ratings, test_size=0.2)

In [15]:
n_users = int(ratings.user_id.nunique())
n_movies = int(ratings.movie_id.nunique())
n_users_train = int(ratings_train.user_id.nunique())
n_movies_train = int(ratings_train.movie_id.nunique())
print(n_users, n_movies, n_users_train, n_movies_train)

943 1682 943 1644


In [16]:
n_latent_factors = 3

In [17]:
# l2_reg = l2(0.00025)
l2_reg = l2(0.00)
movie_input = Input(shape=[1], name='Item')
movie_embedding = Embedding(n_movies + 1, 
                            n_latent_factors, 
                            embeddings_regularizer = l2(0.001),
                            name='Movie-Embedding')(movie_input)
movie_vec = Flatten(name='FlattenMovies')(movie_embedding)

m_biases = Flatten(name='movie_biases_flt')(Embedding(n_movies + 1, 1, name="movie_biases", embeddings_regularizer = l2_reg)(movie_input))

user_input = Input(shape=[1],name='User')
user_vec = Flatten(name='FlattenUsers')(Embedding(n_users + 1, n_latent_factors,embeddings_regularizer = l2_reg,name='User-Embedding')(user_input))
u_biases = Flatten(name='user_biases_flt')(Embedding(n_users + 1, 1, name="user_biases", embeddings_regularizer = l2_reg)(user_input))

In [18]:
prod = Dot(axes=1, name='DotProduct')([movie_vec, user_vec])
out = Add()([prod, u_biases, m_biases])
model = Model([user_input, movie_input], out)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Item (InputLayer)   │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ User (InputLayer)   │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Movie-Embedding     │ (None, 1, 3)      │      5,049 │ Item[0][0]        │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ User-Embedding      │ (None, 1, 3)      │      2,832 │ User[0][0]        │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ FlattenMovies       │ (None, 3)         │          0 │ Movie-Embedding[… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ FlattenUsers        │ (None, 3)         │          0 │ User-Embedding[0… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_biases         │ (None, 1, 1)      │        944 │ User[0][0]        │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ movie_biases        │ (None, 1, 1)      │      1,683 │ Item[0][0]        │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ DotProduct (Dot)    │ (None, 1)         │          0 │ FlattenMovies[0]… │
│                     │                   │            │ FlattenUsers[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_biases_flt     │ (None, 1)         │          0 │ user_biases[0][0] │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ movie_biases_flt    │ (None, 1)         │          0 │ movie_biases[0][… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 1)         │          0 │ DotProduct[0][0], │
│                     │                   │            │ user_biases_flt[… │
│                     │                   │            │ movie_biases_flt… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 10,508 (41.05 KB)

 Trainable params: 10,508 (41.05 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [20]:
model.compile(Adam(learning_rate=0.001), 'mean_squared_error', metrics=[root_mean_squared_error])

In [21]:
history = model.fit([ratings_train.user_id.astype(np.float32), ratings_train.movie_id.astype(np.float32)], 
                    ratings_train.rating.astype(np.float32), 
                    batch_size=320,
                    validation_data=([ratings_val.user_id.astype(np.float32), ratings_val.movie_id.astype(np.float32)], ratings_val.rating.astype(np.float32)), 
                    epochs=100, 
                    #callbacks = [plot_losses],
                    verbose=1)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 13.2676 - root_mean_squared_error: 3.6413 - val_loss: 11.8586 - val_root_mean_squared_error: 3.4416
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 724us/step - loss: 11.1946 - root_mean_squared_error: 3.3406 - val_loss: 9.0296 - val_root_mean_squared_error: 2.9816
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 721us/step - loss: 8.2245 - root_mean_squared_error: 2.8322 - val_loss: 6.1703 - val_root_mean_squared_error: 2.4098
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 709us/step - loss: 5.5754 - root_mean_squared_error: 2.2681 - val_loss: 4.2772 - val_root_mean_squared_error: 1.9225
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 762us/step - loss: 3.9153 - root_mean_squared_error: 1.8130 - val_loss: 3.2984 - val_root_mean_squared_error: 1.6010
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 737us/step - loss: 3.0704 - root_mean_squared_error: 1.5207 - val_loss: 2.8217 - val_root_mean_squared_error: 1.4181
Epoch 7/100
250/250 ━━━━━━━━━━━━━

In [22]:
model.evaluate([ratings_val.user_id.astype(np.float32), ratings_val.movie_id.astype(np.float32)], ratings_val.rating.astype(np.float32))

625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 405us/step - loss: 0.8822 - root_mean_squared_error: 0.9162


[0.8664581179618835, 0.9075854420661926]

In [23]:
movie_embeddings_layer = model.layers[2]
user_embeddings_layer = model.layers[3]

movie_embeddings_layer.name, user_embeddings_layer.name

('Movie-Embedding', 'User-Embedding')

#### Hay una diferencia de 1 entre n_movies, n_users y  el shape de las matrices de embeddigs

In [24]:
movie_embeddings_matrix = movie_embeddings_layer.get_weights()[0]
user_embeddings_matrix = user_embeddings_layer.get_weights()[0]

movie_embeddings_matrix.shape, user_embeddings_matrix.shape, n_movies, n_users

((1683, 3), (944, 3), 1682, 943)

In [25]:
np.save('../data/vector_db/movie_embeddings_matrix.npy', movie_embeddings_matrix)
np.save('../data/vector_db/user_embeddings_matrix.npy', user_embeddings_matrix)
np.save('../data/vector_db/user2Idx.npy', user2Idx)
np.save('../data/vector_db/movie2Idx.npy', movie2Idx)